## Question 6 - Before Super Bowl

In [2]:
import numpy as np
import statsmodels.api as sm
import json
import datetime, time
import collections
from sklearn import metrics
import pytz

hashtags = ['gohawks', 'gopatriots' , 'nfl', 'patriots' , 'sb49', 'superbowl']
results = []
pst_tz = pytz.timezone('America/Los_Angeles')

for ht in hashtags:
    print(ht)
    tweets = open('tweets_#' + ht + '.txt', 'r', encoding="utf8")
    line = json.loads(tweets.readline())
    tweets.seek(0, 0)
    start_time = line.get('citation_date')
    number_of_tweets = 0
    number_of_retweets = 0
    number_of_followers = 0
    max_followers = 0
    piece_time = datetime.datetime(2015, 2, 1, 8, 0).timestamp()
    piece_time_pst = datetime.datetime.fromtimestamp(piece_time, pst_tz)
    
    hour_window = 1 # 1 hour window
    end_time = start_time + hour_window * 3600
    #print(piece_time_pst)


    X = []
    Xi = [] # i => ith window
    Xi.append(1)
    Y = []
    tweet_time_array = []
    
    for tweet in tweets:
            t = json.loads(tweet)
            tweet_time = t.get('firstpost_date')
            tweet_time_pst = datetime.datetime.fromtimestamp(tweet_time, pst_tz)
            if(tweet_time_pst < piece_time_pst):
                tweet_time_array.append(tweet_time_pst)
                if t.get('firstpost_date') < end_time:
                    number_of_tweets = number_of_tweets + 1
                    number_of_retweets = number_of_retweets + t.get('metrics').get('citations').get('total')
                    number_of_followers = number_of_followers + t.get('tweet').get('user').get('followers_count')
                    if t.get('tweet').get('user').get('followers_count') > max_followers:
                        max_followers = t.get('tweet').get('user').get('followers_count')
                else:
                    Xi.append(number_of_tweets)
                    Y.append(number_of_tweets)
                    number_of_tweets = 1           
    
                    Xi.append(number_of_retweets)
                    number_of_retweets = t.get('metrics').get('citations').get('total')
                    Xi.append(number_of_followers)
                    number_of_followers = t.get('tweet').get('user').get('followers_count')
                    Xi.append(max_followers)
                    max_followers = t.get('tweet').get('user').get('followers_count')
                    Xi.append(int(datetime.datetime.fromtimestamp(t.get('firstpost_date')).strftime("%H")))

                    end_time = end_time + hour_window * 3600
                    X.append(Xi)
                    Xi = []
                    Xi.append(1)       

    Y = collections.deque(Y)
    Y.rotate(-1)
    Y = list(Y)

    res = sm.OLS(Y, X).fit()
    results.append(res)   
  
    
    Y_pred = res.predict()
    
    Y_pred = collections.deque(Y_pred)
    Y_pred.rotate(-1)
    Y_pred = list(Y_pred)
    
    # calculate MAE, MSE, RMSE
    print("+++++++++++++++++++++++++++++++++")
    #print("Mean Absolute Error", metrics.mean_absolute_error(Y, Y_pred))
    print("Mean Square Error", metrics.mean_squared_error(Y, Y_pred))
    #print("Root Mean-Square Error" , np.sqrt(metrics.mean_squared_error(Y, Y_pred)))
    print("+++++++++++++++++++++++++++++++++")
    print(max(tweet_time_array))
    #print(res.summary())
    #print('Parameters: ', res.params)
    #print('Standard errors: ', res.bse)
    #print('Predicted values: ', res.predict())
    #print('t_test: ', res.t_test)
    #print('p values: ', res.pvalues)
    print('R-squared', res.rsquared)
    print("********************************")
    print("********************************")    
    tweets.close()  
       

gohawks
+++++++++++++++++++++++++++++++++
Mean Square Error 552718.3379155113
+++++++++++++++++++++++++++++++++
2015-02-01 07:59:44-08:00
R-squared 0.3502532148886828
********************************
********************************
gopatriots
+++++++++++++++++++++++++++++++++
Mean Square Error 2448.8060511435224
+++++++++++++++++++++++++++++++++
2015-02-01 07:59:26-08:00
R-squared 0.7016709273553065
********************************
********************************
nfl
+++++++++++++++++++++++++++++++++
Mean Square Error 236520.92790360673
+++++++++++++++++++++++++++++++++
2015-02-01 07:59:57-08:00
R-squared 0.10999357798420617
********************************
********************************
patriots
+++++++++++++++++++++++++++++++++
Mean Square Error 174922.82561423106
+++++++++++++++++++++++++++++++++
2015-02-01 07:59:45-08:00
R-squared 0.5403952873529696
********************************
********************************
sb49
+++++++++++++++++++++++++++++++++
Mean Square Error 3338.19

## Question 6 -  During Superbowl

In [12]:
 import numpy as np
import statsmodels.api as sm
import json
import datetime, time
import collections
from sklearn import metrics
import pytz

hashtags = ['gohawks', 'gopatriots' , 'nfl', 'patriots' , 'sb49', 'superbowl']
results = []
pst_tz = pytz.timezone('America/Los_Angeles')

for ht in hashtags:
    print(ht)
    tweets = open('tweets_#' + ht + '.txt', 'r', encoding="utf8")
    line = json.loads(tweets.readline())
    tweets.seek(0, 0)
    start_time = line.get('firstpost_date')
    number_of_tweets = 0
    number_of_retweets = 0
    number_of_followers = 0
    max_followers = 0
    piece_time = datetime.datetime(2015, 2, 1, 8, 0).timestamp()
    piece_time_pst = datetime.datetime.fromtimestamp(piece_time, pst_tz)
    piece_time_2 = datetime.datetime(2015, 2, 1, 20, 0).timestamp()
    piece_time_2_pst = datetime.datetime.fromtimestamp(piece_time_2, pst_tz)
    
    hour_window = 0.0833 # 5 min window
    end_time = start_time + hour_window * 3600
    #print(piece_time_pst)


    X = []
    Xi = [] # i => ith window
    Xi.append(1)
    Y = []
    tweet_time_array = []
    
    for tweet in tweets:
            t = json.loads(tweet)
            tweet_time = t.get('firstpost_date')
            tweet_time_pst = datetime.datetime.fromtimestamp(tweet_time, pst_tz)
            if(tweet_time_pst > piece_time_pst and tweet_time_pst < piece_time_2_pst):
                tweet_time_array.append(tweet_time_pst)
                if t.get('firstpost_date') < end_time:
                    number_of_tweets = number_of_tweets + 1
                    number_of_retweets = number_of_retweets + t.get('metrics').get('citations').get('total')
                    number_of_followers = number_of_followers + t.get('tweet').get('user').get('followers_count')
                    if t.get('tweet').get('user').get('followers_count') > max_followers:
                        max_followers = t.get('tweet').get('user').get('followers_count')
                else:
                    Xi.append(number_of_tweets)
                    Y.append(number_of_tweets)
                    number_of_tweets = 1           
    
                    Xi.append(number_of_retweets)
                    number_of_retweets = t.get('metrics').get('citations').get('total')
                    Xi.append(number_of_followers)
                    number_of_followers = t.get('tweet').get('user').get('followers_count')
                    Xi.append(max_followers)
                    max_followers = t.get('tweet').get('user').get('followers_count')
                    Xi.append(int(datetime.datetime.fromtimestamp(t.get('firstpost_date')).strftime("%H")))

                    end_time = end_time + hour_window * 3600
                    X.append(Xi)
                    Xi = []
                    Xi.append(1)       

    Y = collections.deque(Y)
    Y.rotate(-1)
    Y = list(Y)

    res = sm.OLS(Y, X).fit()
    results.append(res)   
  
    
    Y_pred = res.predict()
    
    Y_pred = collections.deque(Y_pred)
    Y_pred.rotate(-1)
    Y_pred = list(Y_pred)
    
    # calculate MAE, MSE, RMSE
    print("+++++++++++++++++++++++++++++++++")
    #print("Mean Absolute Error", metrics.mean_absolute_error(Y, Y_pred))
    print("Mean Square Error", metrics.mean_squared_error(Y, Y_pred))
    #print("Root Mean-Square Error" , np.sqrt(metrics.mean_squared_error(Y, Y_pred)))
    print("+++++++++++++++++++++++++++++++++")
    print(max(tweet_time_array))
    #print(res.summary())
    #print('Parameters: ', res.params)
    #print('Standard errors: ', res.bse)
    #print('Predicted values: ', res.predict())
    #print('t_test: ', res.t_test)
    #print('p values: ', res.pvalues)
    print('R-squared', res.rsquared)
    print("********************************")
    print("********************************")    
    tweets.close()  

gohawks
+++++++++++++++++++++++++++++++++
Mean Square Error 128.38383764108983
+++++++++++++++++++++++++++++++++
2015-02-01 19:59:59-08:00
R-squared 0.7591374868572406
********************************
********************************
gopatriots
+++++++++++++++++++++++++++++++++
Mean Square Error 107.814934628869
+++++++++++++++++++++++++++++++++
2015-02-01 19:59:33-08:00
R-squared 0.4126035191406431
********************************
********************************
nfl
+++++++++++++++++++++++++++++++++
Mean Square Error 30.31551526379542
+++++++++++++++++++++++++++++++++
2015-02-01 19:59:58-08:00
R-squared 0.8751759312076559
********************************
********************************
patriots
+++++++++++++++++++++++++++++++++
Mean Square Error 2974.2977943074525
+++++++++++++++++++++++++++++++++
2015-02-01 19:59:57-08:00
R-squared 0.8721084712481884
********************************
********************************
sb49
+++++++++++++++++++++++++++++++++
Mean Square Error 4023.21284

In [16]:
print(min(tweet_time_array))
print(max(tweet_time_array))
print(X)

2015-02-01 08:00:01-08:00
2015-02-01 19:59:59-08:00
[[1, 0, 0, 0, 0, 8], [1, 1, 9, 477801, 477801, 8], [1, 1, 1, 3038, 3038, 8], [1, 1, 4, 53360, 53360, 8], [1, 1, 34, 16405, 16405, 8], [1, 1, 1, 49, 49, 8], [1, 1, 1, 3614, 3614, 8], [1, 1, 1, 511, 511, 8], [1, 1, 1, 77, 77, 8], [1, 1, 1, 21, 21, 8], [1, 1, 1, 13, 13, 8], [1, 1, 1, 2234, 2234, 8], [1, 1, 1, 1272, 1272, 8], [1, 1, 1, 72, 72, 8], [1, 1, 2, 13317, 13317, 8], [1, 1, 1, 309129, 309129, 8], [1, 1, 1, 6264, 6264, 8], [1, 1, 1, 1330, 1330, 8], [1, 1, 1, 255, 255, 8], [1, 1, 1, 265, 265, 8], [1, 1, 1, 1383, 1383, 8], [1, 1, 1, 1898, 1898, 8], [1, 1, 1, 621, 621, 8], [1, 1, 1, 1130, 1130, 8], [1, 1, 2, 10204, 10204, 8], [1, 1, 5, 503, 503, 8], [1, 1, 2, 16314, 16314, 8], [1, 1, 1, 3137, 3137, 8], [1, 1, 2, 7493, 7493, 8], [1, 1, 1, 221, 221, 8], [1, 1, 1, 1503, 1503, 8], [1, 1, 1, 53, 53, 8], [1, 1, 1, 81, 81, 8], [1, 1, 3, 155, 155, 8], [1, 1, 1, 1076, 1076, 8], [1, 1, 5, 341357, 341357, 8], [1, 1, 2, 89990, 89990, 8], [1, 1, 1

## Question 6 - After Superbowl

In [15]:
import numpy as np
import statsmodels.api as sm
import json
import datetime, time
import collections
from sklearn import metrics
import pytz

hashtags = ['gohawks', 'gopatriots' , 'nfl', 'patriots' , 'sb49', 'superbowl']
results = []
pst_tz = pytz.timezone('America/Los_Angeles')

for ht in hashtags:
    print(ht)
    tweets = open('tweets_#' + ht + '.txt', 'r', encoding="utf8")
    line = json.loads(tweets.readline())
    tweets.seek(0, 0)
    start_time = line.get('firstpost_date')
    number_of_tweets = 0
    number_of_retweets = 0
    number_of_followers = 0
    max_followers = 0
    piece_time = datetime.datetime(2015, 2, 1, 20, 0).timestamp()
    piece_time_pst = datetime.datetime.fromtimestamp(piece_time, pst_tz)
    
    hour_window = 1 # 1 hour window
    end_time = start_time + hour_window * 3600
    #print(piece_time_pst)


    X = []
    Xi = [] # i => ith window
    Xi.append(1)
    Y = []
    tweet_time_array = []
    
    for tweet in tweets:
            t = json.loads(tweet)
            tweet_time = t.get('firstpost_date')
            tweet_time_pst = datetime.datetime.fromtimestamp(tweet_time, pst_tz)
            if(tweet_time_pst > piece_time_pst):
                tweet_time_array.append(tweet_time_pst)
                if t.get('firstpost_date') < end_time:
                    number_of_tweets = number_of_tweets + 1
                    number_of_retweets = number_of_retweets + t.get('metrics').get('citations').get('total')
                    number_of_followers = number_of_followers + t.get('tweet').get('user').get('followers_count')
                    if t.get('tweet').get('user').get('followers_count') > max_followers:
                        max_followers = t.get('tweet').get('user').get('followers_count')
                else:
                    Xi.append(number_of_tweets)
                    Y.append(number_of_tweets)
                    number_of_tweets = 1           
    
                    Xi.append(number_of_retweets)
                    number_of_retweets = t.get('metrics').get('citations').get('total')
                    Xi.append(number_of_followers)
                    number_of_followers = t.get('tweet').get('user').get('followers_count')
                    Xi.append(max_followers)
                    max_followers = t.get('tweet').get('user').get('followers_count')
                    Xi.append(int(datetime.datetime.fromtimestamp(t.get('firstpost_date')).strftime("%H")))

                    end_time = end_time + hour_window * 3600
                    X.append(Xi)
                    Xi = []
                    Xi.append(1)       

    Y = collections.deque(Y)
    Y.rotate(-1)
    Y = list(Y)

    res = sm.OLS(Y, X).fit()
    results.append(res)   
  
    
    Y_pred = res.predict()
    
    Y_pred = collections.deque(Y_pred)
    Y_pred.rotate(-1)
    Y_pred = list(Y_pred)
    
    # calculate MAE, MSE, RMSE
    print("+++++++++++++++++++++++++++++++++")
    #print("Mean Absolute Error", metrics.mean_absolute_error(Y, Y_pred))
    print("Mean Square Error", metrics.mean_squared_error(Y, Y_pred))
    #print("Root Mean-Square Error" , np.sqrt(metrics.mean_squared_error(Y, Y_pred)))
    print("+++++++++++++++++++++++++++++++++")
    print(max(tweet_time_array))
    #print(res.summary())
    #print('Parameters: ', res.params)
    #print('Standard errors: ', res.bse)
    #print('Predicted values: ', res.predict())
    #print('t_test: ', res.t_test)
    #print('p values: ', res.pvalues)
    print('R-squared', res.rsquared)
    print("********************************")
    print("********************************")    
    tweets.close()  

gohawks
+++++++++++++++++++++++++++++++++
Mean Square Error 475.553897396
+++++++++++++++++++++++++++++++++
2015-02-07 02:17:49-08:00
R-squared 0.886171030062
********************************
********************************
gopatriots
+++++++++++++++++++++++++++++++++
Mean Square Error 0.501694582664
+++++++++++++++++++++++++++++++++
2015-02-06 23:54:35-08:00
R-squared 0.74459877235
********************************
********************************
nfl
+++++++++++++++++++++++++++++++++
Mean Square Error 271.984684246
+++++++++++++++++++++++++++++++++
2015-02-07 10:55:36-08:00
R-squared 0.935036299373
********************************
********************************
patriots
+++++++++++++++++++++++++++++++++
Mean Square Error 1183.02295956
+++++++++++++++++++++++++++++++++
2015-02-07 10:55:00-08:00
R-squared 0.657863598339
********************************
********************************
sb49
+++++++++++++++++++++++++++++++++
Mean Square Error 4678.94631772
+++++++++++++++++++++++++++++

In [10]:
print(min(tweet_time_array))
print(max(tweet_time_array))

2015-02-01 08:00:01-08:00
2015-02-01 16:37:34-08:00


## Q7 Aggregated Data - Before Super Bowl

In [17]:
import numpy as np
import statsmodels.api as sm
import json
import datetime, time
import collections
from sklearn import metrics
import pytz

hashtags = ['gohawks', 'gopatriots' , 'nfl', 'patriots' , 'sb49', 'superbowl']
results = []
pst_tz = pytz.timezone('America/Los_Angeles')

number_of_tweets = 0
number_of_retweets = 0
number_of_followers = 0
max_followers = 0

X = []
Xi = [] # i => ith window
Xi.append(1)
Y = []
tweet_time_array = []



for ht in hashtags:
    print(ht)
    tweets = open('tweets_#' + ht + '.txt', 'r', encoding="utf8")
    line = json.loads(tweets.readline())
    tweets.seek(0, 0)
    start_time = line.get('firstpost_date')
    
    piece_time = datetime.datetime(2015, 2, 1, 8, 0).timestamp()
    piece_time_pst = datetime.datetime.fromtimestamp(piece_time, pst_tz)
    
    hour_window = 1 # 1 hour window
    end_time = start_time + hour_window * 3600
    #print(piece_time_pst)



    
    for tweet in tweets:
            t = json.loads(tweet)
            tweet_time = t.get('firstpost_date')
            tweet_time_pst = datetime.datetime.fromtimestamp(tweet_time, pst_tz)
            if(tweet_time_pst < piece_time_pst):
                tweet_time_array.append(tweet_time_pst)
                if t.get('firstpost_date') < end_time:
                    number_of_tweets = number_of_tweets + 1
                    number_of_retweets = number_of_retweets + t.get('metrics').get('citations').get('total')
                    number_of_followers = number_of_followers + t.get('tweet').get('user').get('followers_count')
                    if t.get('tweet').get('user').get('followers_count') > max_followers:
                        max_followers = t.get('tweet').get('user').get('followers_count')
                else:
                    Xi.append(number_of_tweets)
                    Y.append(number_of_tweets)
                    number_of_tweets = 1           
    
                    Xi.append(number_of_retweets)
                    number_of_retweets = t.get('metrics').get('citations').get('total')
                    Xi.append(number_of_followers)
                    number_of_followers = t.get('tweet').get('user').get('followers_count')
                    Xi.append(max_followers)
                    max_followers = t.get('tweet').get('user').get('followers_count')
                    Xi.append(int(datetime.datetime.fromtimestamp(t.get('firstpost_date')).strftime("%H")))

                    end_time = end_time + hour_window * 3600
                    X.append(Xi)
                    Xi = []
                    Xi.append(1)       

Y = collections.deque(Y)
Y.rotate(-1)
Y = list(Y)

res = sm.OLS(Y, X).fit()
results.append(res)   
  
    
Y_pred = res.predict()
    
Y_pred = collections.deque(Y_pred)
Y_pred.rotate(-1)
Y_pred = list(Y_pred)
    
# calculate MAE, MSE, RMSE
print("+++++++++++++++++++++++++++++++++")
#print("Mean Absolute Error", metrics.mean_absolute_error(Y, Y_pred))
print("Mean Square Error", metrics.mean_squared_error(Y, Y_pred))
#print("Root Mean-Square Error" , np.sqrt(metrics.mean_squared_error(Y, Y_pred)))
print("+++++++++++++++++++++++++++++++++")
print(max(tweet_time_array))
#print(res.summary())
#print('Parameters: ', res.params)
#print('Standard errors: ', res.bse)
#print('Predicted values: ', res.predict())
#print('t_test: ', res.t_test)
#print('p values: ', res.pvalues)
print('R-squared', res.rsquared)
print("********************************")
print("********************************")    
tweets.close()  
       

gohawks
gopatriots
nfl
patriots
sb49
superbowl
+++++++++++++++++++++++++++++++++
Mean Square Error 48783.28915909727
+++++++++++++++++++++++++++++++++
2015-02-01 07:59:59-08:00
R-squared 0.4130508267944476
********************************
********************************


## Aggregated Data - During Super Bowl (5 mins window)

In [18]:
 import numpy as np
import statsmodels.api as sm
import json
import datetime, time
import collections
from sklearn import metrics
import pytz

hashtags = ['gohawks', 'gopatriots' , 'nfl', 'patriots' , 'sb49', 'superbowl']
results = []
pst_tz = pytz.timezone('America/Los_Angeles')
number_of_tweets = 0
number_of_retweets = 0
number_of_followers = 0
max_followers = 0

X = []
Xi = [] # i => ith window
Xi.append(1)
Y = []
tweet_time_array = []

for ht in hashtags:
    print(ht)
    tweets = open('tweets_#' + ht + '.txt', 'r', encoding="utf8")
    line = json.loads(tweets.readline())
    tweets.seek(0, 0)
    start_time = line.get('firstpost_date')

    piece_time = datetime.datetime(2015, 2, 1, 8, 0).timestamp()
    piece_time_pst = datetime.datetime.fromtimestamp(piece_time, pst_tz)
    piece_time_2 = datetime.datetime(2015, 2, 1, 20, 0).timestamp()
    piece_time_2_pst = datetime.datetime.fromtimestamp(piece_time_2, pst_tz)
    
    hour_window = 0.0833 # 5 min window
    end_time = start_time + hour_window * 3600
    #print(piece_time_pst)



    
    for tweet in tweets:
            t = json.loads(tweet)
            tweet_time = t.get('firstpost_date')
            tweet_time_pst = datetime.datetime.fromtimestamp(tweet_time, pst_tz)
            if(tweet_time_pst > piece_time_pst and tweet_time_pst < piece_time_2_pst):
                tweet_time_array.append(tweet_time_pst)
                if t.get('firstpost_date') < end_time:
                    number_of_tweets = number_of_tweets + 1
                    number_of_retweets = number_of_retweets + t.get('metrics').get('citations').get('total')
                    number_of_followers = number_of_followers + t.get('tweet').get('user').get('followers_count')
                    if t.get('tweet').get('user').get('followers_count') > max_followers:
                        max_followers = t.get('tweet').get('user').get('followers_count')
                else:
                    Xi.append(number_of_tweets)
                    Y.append(number_of_tweets)
                    number_of_tweets = 1           
    
                    Xi.append(number_of_retweets)
                    number_of_retweets = t.get('metrics').get('citations').get('total')
                    Xi.append(number_of_followers)
                    number_of_followers = t.get('tweet').get('user').get('followers_count')
                    Xi.append(max_followers)
                    max_followers = t.get('tweet').get('user').get('followers_count')
                    Xi.append(int(datetime.datetime.fromtimestamp(t.get('firstpost_date')).strftime("%H")))

                    end_time = end_time + hour_window * 3600
                    X.append(Xi)
                    Xi = []
                    Xi.append(1)       

Y = collections.deque(Y)
Y.rotate(-1)
Y = list(Y)

res = sm.OLS(Y, X).fit()
results.append(res)   


Y_pred = res.predict()

Y_pred = collections.deque(Y_pred)
Y_pred.rotate(-1)
Y_pred = list(Y_pred)

# calculate MAE, MSE, RMSE
print("+++++++++++++++++++++++++++++++++")
#print("Mean Absolute Error", metrics.mean_absolute_error(Y, Y_pred))
print("Mean Square Error", metrics.mean_squared_error(Y, Y_pred))
#print("Root Mean-Square Error" , np.sqrt(metrics.mean_squared_error(Y, Y_pred)))
print("+++++++++++++++++++++++++++++++++")
print(max(tweet_time_array))
#print(res.summary())
#print('Parameters: ', res.params)
#print('Standard errors: ', res.bse)
#print('Predicted values: ', res.predict())
#print('t_test: ', res.t_test)
#print('p values: ', res.pvalues)
print('R-squared', res.rsquared)
print("********************************")
print("********************************")    
tweets.close()  

gohawks
gopatriots
nfl
patriots
sb49
superbowl
+++++++++++++++++++++++++++++++++
Mean Square Error 4558.195379662992
+++++++++++++++++++++++++++++++++
2015-02-01 19:59:59-08:00
R-squared 0.9329007129298981
********************************
********************************


## After Super Bowl - With Aggregated Data

In [17]:
import numpy as np
import statsmodels.api as sm
import json
import datetime, time
import collections
from sklearn import metrics
import pytz

hashtags = ['gohawks', 'gopatriots' , 'nfl', 'patriots' , 'sb49', 'superbowl']
results = []
pst_tz = pytz.timezone('America/Los_Angeles')
number_of_tweets = 0
number_of_retweets = 0
number_of_followers = 0
max_followers = 0
X = []
Xi = [] # i => ith window
Xi.append(1)
Y = []
tweet_time_array = []

for ht in hashtags:
    print(ht)
    tweets = open('tweets_#' + ht + '.txt', 'r', encoding="utf8")
    line = json.loads(tweets.readline())
    tweets.seek(0, 0)
    start_time = line.get('firstpost_date')
    ## Time Filtering after 1st Feb 8pm PST
    piece_time = datetime.datetime(2015, 2, 1, 20, 0).timestamp()
    piece_time_pst = datetime.datetime.fromtimestamp(piece_time, pst_tz)
    
    hour_window = 1 # 1 hour window
    end_time = start_time + hour_window * 3600
    #print(piece_time_pst)

    for tweet in tweets:
            t = json.loads(tweet)
            tweet_time = t.get('firstpost_date')
            tweet_time_pst = datetime.datetime.fromtimestamp(tweet_time, pst_tz)
            if(tweet_time_pst > piece_time_pst):
                tweet_time_array.append(tweet_time_pst)
                if t.get('firstpost_date') < end_time:
                    number_of_tweets = number_of_tweets + 1
                    number_of_retweets = number_of_retweets + t.get('metrics').get('citations').get('total')
                    number_of_followers = number_of_followers + t.get('tweet').get('user').get('followers_count')
                    if t.get('tweet').get('user').get('followers_count') > max_followers:
                        max_followers = t.get('tweet').get('user').get('followers_count')
                else:
                    Xi.append(number_of_tweets)
                    Y.append(number_of_tweets)
                    number_of_tweets = 1           
    
                    Xi.append(number_of_retweets)
                    number_of_retweets = t.get('metrics').get('citations').get('total')
                    Xi.append(number_of_followers)
                    number_of_followers = t.get('tweet').get('user').get('followers_count')
                    Xi.append(max_followers)
                    max_followers = t.get('tweet').get('user').get('followers_count')
                    Xi.append(int(datetime.datetime.fromtimestamp(t.get('firstpost_date')).strftime("%H")))

                    end_time = end_time + hour_window * 3600
                    X.append(Xi)
                    Xi = []
                    Xi.append(1)       

Y = collections.deque(Y)
Y.rotate(-1)
Y = list(Y)

res = sm.OLS(Y, X).fit()
results.append(res)   
  
    
Y_pred = res.predict()
    
Y_pred = collections.deque(Y_pred)
Y_pred.rotate(-1)
Y_pred = list(Y_pred)
    
# calculate MAE, MSE, RMSE
print("+++++++++++++++++++++++++++++++++")
#print("Mean Absolute Error", metrics.mean_absolute_error(Y, Y_pred))
print("Mean Square Error", metrics.mean_squared_error(Y, Y_pred))
#print("Root Mean-Square Error" , np.sqrt(metrics.mean_squared_error(Y, Y_pred)))
print("+++++++++++++++++++++++++++++++++")
print(max(tweet_time_array))
#print(res.summary())
#print('Parameters: ', res.params)
#print('Standard errors: ', res.bse)
#print('Predicted values: ', res.predict())
#print('t_test: ', res.t_test)
#print('p values: ', res.pvalues)
print('R-squared', res.rsquared)
print("********************************")
print("********************************")    
tweets.close()  

gohawks
gopatriots
nfl
patriots
sb49
superbowl
+++++++++++++++++++++++++++++++++
Mean Square Error 663.301187189
+++++++++++++++++++++++++++++++++
2015-02-07 10:55:36-08:00
R-squared 0.838813693887
********************************
********************************
